# Reproduce

In [1]:
# OK
# import packages

import os 
import regex as re
import numpy as np 
from matplotlib import pyplot as plt, gridspec as gridspec # local: 3.4.1

In [2]:
# OK

# enum patterns
pat_ts = "ts:([0-9]+\.[0-9]+)"
pat_rtt = "time=([0-9]+[\.0-9]*)\sms"
pat_cwnd = "cwnd:([0-9]+[\.0-9]*)"
pat_qlen = "qlen:([0-9]+[\.0-9]*)"

va_type = ['cwnd', 'qlen', 'rtt']
al_type = ['taildrop', 'red', 'codel']

In [3]:
# OK

# Extract cwnd/rtt/qlen 
def extract_txt_for_rp(data_dir, pat):
    '''
    usage: pat = 'cwnd' or 'qlen' or 'rtt'
    '''
    path = "./"
    if pat == va_type[0]:
        path  += data_dir + "/" + "cwnd.txt"
        pat_va = pat_cwnd
    elif pat == va_type[1]:
        path += data_dir + "/" + "qlen.txt"
        pat_va = pat_qlen
    elif pat == va_type[2]:
        path += data_dir + "/" + "rtt.txt"
        pat_va = pat_rtt
    else:
        print("Usage: extract_txt_for_rp(data_dir, pat), pat = 'cwnd'/'rtt'/'qlen'")

    l_ts = []
    l_va = []
    base_ts = 0
    flag = 1
    with open(path, 'r') as file:
        logs = file.readlines()
        for log in logs:
            # pattern match
            ts_match_list = re.findall(pat_ts, log)
            va_match_list = re.findall(pat_va, log)
            if len(va_match_list) < 1:
                continue
            if len(ts_match_list) < 1:
                break
            
            # store relative time stamp
            ts = float(ts_match_list[0]) - base_ts
            if flag==1 :
                base_ts = ts
                ts = 0
                flag = 0
            if ts>60:
                break
            
            # store va value
            if pat == va_type[2]: 
                va = float(va_match_list[0])
            else:
                va= int(va_match_list[0])
            
            l_ts.append(ts)
            l_va.append(va)

    # test or true return
    return l_ts, l_va


In [4]:
# OK
# Function: plot for [ts, va], ts = time stamp = x, va = y

def plot_for_rp(data_dir, pat):
    '''
    usage: pat = 'cwnd' or 'qlen' or 'rtt'
    '''

    x, y = extract_txt_for_rp(data_dir, pat)

    # color
    if pat == va_type[0]:
        line_color = 'b'
    elif pat == va_type[1]:
        line_color = 'r'
    elif pat == va_type[2]:
        line_color = 'm'

    # axis boundary
    xmin = min(x)
    xmax = max(x)
    ymin = min(y)
    ymax = max(y)
    x_lb = xmin - 0.05*(xmax-xmin)
    x_hb = xmax + 0.05*(xmax-xmin)
    y_lb = ymin - 0.05*(ymax-ymin)
    y_hb = ymax + 0.05*(ymax-ymin)

    # plotting by args
    plt.plot(x, y, 
             color=line_color,
             linewidth=0.5
             )
    plt.axis([x_lb, x_hb, 
              y_lb, y_hb])
    
    # save fig
    fig_path = "./figs_rp/single-%s-%s-fig.png" % (data_dir, pat)
    plt.savefig(fig_path, dpi=800)
    plt.close()


In [5]:
# OK
# plotting and save results

# list folders, select the ones that store the data output
path = '.'
items = os.listdir(path)
data_dirs = []
for item in items:
    # item is directory && 'qlen' in item
    if os.path.isdir(item) and re.match("qlen-.+[0-9]+", item):
        data_dirs.append(item)

# plot figs
for res in data_dirs:
    for type in va_type:
        plot_for_rp(res, type)